In [1]:
import numba
import numpy as np
import tqdm

import warnings
warnings.filterwarnings('ignore')

In [2]:
@numba.njit
def rolling_hashes(xs, n, base, mod):
    assert max(xs) + 1 < base
    
    over = 1
    for _ in range(n):
        over *= base
        over %= mod
    
    hashes = []
    cur = 0
    for x in xs[:n]:
        cur = (base * cur + x + 1) % mod
        
    if n <= len(xs):
        hashes.append(cur)
        for i, x in enumerate(xs[n:]):
            cur = (base * cur + x + 1) % mod
            cur -= (xs[i] + 1) * over
            cur %= mod
            hashes.append(cur)
    
    return hashes

@numba.njit
def super_rolling_hashes(xs, n, p):
    MODS = [1000000007]#, 1000000009]#, 1000000021, 1000000033]
    BASE = p + 1
    
    hashes = []
    for mod in MODS:
        hashes.append(rolling_hashes(xs=xs, n=n, base=BASE, mod=mod))
                
    return hashes
        

def count_sequences(p, n):
    seq_hashes = set()
    row = [1]
    
    ub = 1
    while ub <= n:
        ub *= p
    ub *= p
    if p == 2:
        ub *= 2
    
    for _ in tqdm.trange(ub):
        for super_hash in zip(*super_rolling_hashes(xs=row, n=n, p=p)):
            seq_hashes.add(super_hash)
        
        prv = 0
        for i in range(len(row)):
            tmp = row[i]
            row[i] = (row[i] + prv) % p
            prv = tmp
        row.append(1)
            
    return len(seq_hashes)

count_sequences(p=2, n=7)

100%|██████████| 32/32 [00:00<00:00, 35.04it/s]


44

In [3]:
def get_poly(p):
    xs, ys = [], []
    for k in range(3):    
        n = p ** k
        cnt = count_sequences(p=p, n=n)

        x, y = n, cnt
        xs.append(x)
        ys.append(y)

    return np.poly1d(np.polyfit(xs, ys, deg=2))

print(get_poly(p=5))

100%|██████████| 625/625 [00:00<00:00, 1207.92it/s]

    2
12 x - 10 x + 3


In [4]:
%%time
for p in [13]:
    print(get_poly(p))

100%|██████████| 28561/28561 [15:14<00:00, 31.24it/s]


       2
87.31 x - 76.36 x + 2.052
CPU times: user 15min 9s, sys: 6.31 s, total: 15min 16s
Wall time: 15min 20s
